## Decision Tree Regression
___

1. [Introduction](#1.-Introduction) <br><br>
2. [Synthetic data creation](#2.-Synthetic-data-creation) <br>
    2.1 [Generate data](#2.1.-Generate-data) <br>
    2.2 [Compute the data's mean and covariance matrix](#2.2-Compute-the-data's-mean-and-covariance-matrix) <br><br>

3. [Synthetic data creation](#2.-Synthetic-data-creation)<br>
    3.1. [Variance maximization formulation](#3.-Variance-maximization-formulation)<br>
    3.2. [Defining function to be optimized](#3.1-Defining-function-to-be-optimized)<br>
    3.3. [Plotting the data, along with its principal components](#3.3-Plotting-the-data,-along-with-its-principal-components)<br>
    3.4. [Project the data onto the first principal component](#3.4-Project-the-data-onto-u1,-compute-the-variance-and-plot) <br>
    3.5. [[Extra] Plot the variance against different vector directions](3.4-[Extra]-Plot-the-variance-against-different--vector-directions)<br><br>
    
4. [References](#4.-References)<br>
___

### 1. Introdução

<p style="text-align: justify;">  
Árvores de decisão são algoritmos de aprendizado de máquina que predizem uma saída desejada a partir da aplicação de um **conjunto hierárquico de regras** à entrada. Tal saída pode ser tanto um valor numérico, o que leva a problema de regressão, quanto um valor categórico, onde tem-se um problema de classificação. <br><br>

Embora o princípio seja um só, grande parte dos exemplos sobre árvores de decisão volta-se para problemas de classificação. Poucas vezes um problema de regressão é utilizado para ilustrar a intuição da árvore de decisão. 
</p>

Vamos a um exemplo pouco prático, mas de boa ilustração. Imagine que você deseja supor a altura de uma criança   

Um professor de Educação Física reuniu seus alunos num certo dia e mediu a altura de cada um deles, fazendo uma lista com suas alturas e idades. Só depois percebeu que um aluno havia faltado a aula. Sua idade ele facilmente descobriu pelo Facebook. Ele podia simplesmente ter perguntado a altura A idade ele facilmente descobriu no Facebook

anotando  . em um um dado  tenha  uma lista contendo as alturas dos alunos de uma sala de aula, o que te dá uma ideia da faixa de estatura dos alunos, digamos que seja de 1.40m a 1.80m. Infelizmente, quando a lista foi feitaPor motivos inexplicáveis, você foi desafiado a adivinhar a altura de um novo aluno está interessado em  , Tem a se e você está interessado. o Antes de entrar na ideia da árvore de decisão, vamos lembrar da velha busca binária.


<#define MACRO|value>
Use <#MACRO>

In [2]:
# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from ipywidgets import interactive

In [3]:
# Importing the dataset
dataset = pd.read_csv('Position_Salaries.csv')
X = dataset.iloc[:, 1:2].values
y = dataset.iloc[:, 2].values

In [4]:
class RegressionTree():
    
    def __init__(self, min_samples_split = 2):
        self.root = self.Node()
        self.min_samples_split = min_samples_split
        pass
    
    class Node():
        def __init__(self, parent = None):
            self.parent = parent
            self.left_child  = None
            self.right_child = None
            self.threshold   = None
            self.isLeaf      = True
            self.output      = None

    def select_split_point(self, X, y, metric = 'mse'):
        grid = np.arange(np.min(X),np.max(X),0.01*(np.max(X)-np.min(X)))
        left  = [np.mean([(split - x)**2 for x in X if x <= split]) for split in grid]
        right = [np.mean([(split - x)**2 for x in X if x > split]) for split in grid]
        return grid[np.argmin([l+r for l,r in zip(left,right)])]
        
    def split(self, parent, X, y):
            
        #Not a leaf
        parent.isLeaf = False

        #Select the best X split point
        best_split = self.select_split_point(X, y, 'mse')
        
        #Update threshold
        parent.threshold = best_split

        #Create children
        left_child  = self.Node(parent = parent)
        right_child = self.Node(parent = parent)

        #Split data
        X_l, X_r = X[X <= best_split].reshape(-1,1), X[X > best_split].reshape(-1,1)
        y_l, y_r = y[X[:,0] <= best_split], y[X[:,0] > best_split] 

        if len(X_l) >= self.min_samples_split:
            self.split(left_child, X_l, y_l)
        else:
            left_child.output = np.mean(y_l)
        if len(X_r) >= self.min_samples_split:
            self.split(right_child, X_r, y_r)
        else:
            right_child.output = np.mean(y_r)

        parent.left_child = left_child
        parent.right_child = right_child

    def fit(self, X, y):
        #Create root node
        self.split(self.root, X, y)      
    
    def compute(self, node, value):
        if node.isLeaf:
            return node.output
        if value >= node.threshold:
            return self.compute(node.right_child, value)
        else:
            return self.compute( node.left_child, value)
        
    def predict(self, X):
        if not isinstance(X, np.ndarray):
            return self.compute(self.root, X)
        return [self.compute(self.root, x) for x in X]     

    def print_split(self, node, shift_vec):
        
        if node.left_child == node.right_child == None:
            return 'Leaf ' + u'\u2192 ' + 'y = ' + '{0:.0f}'.format(node.output) + '\n'

        shift = shift_vec.copy()
        
        #Right child condition
        right_condition = u'\u252C ' + 'x >= ' + '{0:.2f}'.format(node.threshold) + ': '
        
        #Left child condition
        left_condition  = ""
        for i,s in enumerate(shift):
            left_condition += s[0] + ' '*s[1]
        left_condition += u'\u2514 ' + 'x <  ' + '{0:.2f}'.format(node.threshold) + ': '
        
        #Append shift vector
        shift.append([u'\u2502 ', len(right_condition) - len(u'\u252C ')])
        
        summary = right_condition + self.print_split(node.right_child, shift)        
        shift[-1][0] = '  '
        summary += left_condition + self.print_split(node.left_child, shift)
                
        return summary
        
    def __str__(self):
        summary  = ''
        summary += self.print_split(self.root, [])
        
       
        return summary

In [5]:
tree = RegressionTree(min_samples_split = 2)
tree.fit(X,y)
print(tree)

┬ x >= 5.50: ┬ x >= 8.24: ┬ x >= 9.50: Leaf → y = 1000000
│            │            └ x <  9.50: Leaf → y = 500000
│            └ x <  8.24: ┬ x >= 7.26: Leaf → y = 300000
│                         └ x <  7.26: ┬ x >= 6.50: Leaf → y = 200000
│                                      └ x <  6.50: Leaf → y = 150000
└ x <  5.50: ┬ x >= 3.24: ┬ x >= 4.50: Leaf → y = 110000
             │            └ x <  4.50: Leaf → y = 80000
             └ x <  3.24: ┬ x >= 1.74: ┬ x >= 2.50: Leaf → y = 60000
                          │            └ x <  2.50: Leaf → y = 50000
                          └ x <  1.74: Leaf → y = 45000



In [6]:
def func(min_samples):
    tree = RegressionTree(min_samples_split = min_samples)
    tree.fit(X,y)
    X_grid = np.arange(np.min(X),np.max(X),0.001*(np.max(X)-np.min(X)))
    y_pred = tree.predict(X_grid)
    plt.figure(0)
    plt.scatter(X,y)
    plt.plot(X_grid, y_pred, color = 'r')
    plt.ylim(0, 1.05*max(y))
    plt.show()
    print(tree)

In [7]:
interactive_plot = interactive(func, min_samples = (2, len(X), 1))
interactive_plot